# This is the notebook used for Milestone 1. Please do not modify!
- It is completely self-contained, with all class definitions inside.
- This is kept for posterity and to refer back to a working setup.

In [1]:
import RPi.GPIO as GPIO
import gpiozero
import time
import numpy as np
import warnings
from matplotlib import pyplot as plt
from matplotlib.patches import Arrow
from IPython import display
import cv2
import threading
import random


In [2]:
###################################################### Defining our Constants ######################################################
# Encoder pins
# ML = motor left, MR = motor right
GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

ML_ENC_A = 15 # yellow encoder c 23
ML_ENC_B = 14 # white encoder c 24

MR_ENC_A = 24 # yellow encoder c
MR_ENC_B = 23 # white encoder c

# The number of encoder steps per revolution. This was calculated according to the math that Lacie had in Notion.
MAX_ENC_STEPS = 900

# Motor Pins
ML_IN1 = 17 # IN1 
ML_IN2 = 27 # IN2
ML_ENA = 11 # Used for PWM

MR_IN3 = 22 # IN3
MR_IN4 = 10 # IN4
MR_ENB = 9 # Used for PWM

# Physical dimensions
WHEEL_RADIUS = 0.027 # meters
WHEEL_SEP = 0.229 # meters

# wheel separation measurement (ASCII art)
"""
    <------- l ------->
    |                 |
 ___|___           ___|___
|       |         |       |
|   O   |         |   O   |
|_______|         |_______|
    ^                 ^
    |                 |
  Left wheel       Right wheel
  center            center
"""


/tmp/ipykernel_1344/899567074.py:4: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!
  GPIO.cleanup()


'\n    <------- l ------->\n    |                 |\n ___|___           ___|___\n|       |         |       |\n|   O   |         |   O   |\n|_______|         |_______|\n    ^                 ^\n    |                 |\n  Left wheel       Right wheel\n  center            center\n'

In [3]:
###################################################### Class Definitions ######################################################

# Used to implement smoothing dt. Lower alpha means less smoothing, faster response to changes
class ExponentialMovingAverage:
    def __init__(self, alpha):
        self.alpha = alpha
        self.value = None

    def update(self, new_value):
        if self.value is None:
            self.value = new_value
        else:
            self.value = (1 - self.alpha) * new_value + self.alpha * self.value
        return self.value
    

# Used to implement velocity smoothing. Lower tau means less smoothing, faster response to changes.
class LowPassFilter:
    def __init__(self, tau):
        self.tau = tau
        self.previous_value = None
        self._last_smoothed_value = None

    def update(self, new_value, dt):
        if self.previous_value is None:
            self.previous_value = new_value
            self._last_smoothed_value = new_value
        else:
            alpha = dt / (self.tau + dt)
            self._last_smoothed_value = alpha * new_value + (1 - alpha) * self.previous_value
            self.previous_value = self._last_smoothed_value
        return self._last_smoothed_value

    @property
    def value(self):
        if self._last_smoothed_value is None:
            return 0
        return self._last_smoothed_value
    

class DiffDriveRobot:
    def __init__(
            self, 
            dt=0.05, 
            real_time=False,
            tau = 0.1,
            wheel_radius=WHEEL_RADIUS, 
            wheel_sep=WHEEL_SEP,
            max_enc_steps=MAX_ENC_STEPS,
            ml_pwm=ML_ENA,
            mr_pwm=MR_ENB,
            ml_in1=ML_IN1,
            ml_in2=ML_IN2,
            mr_in3=MR_IN3,
            mr_in4=MR_IN4,
            ml_encA=ML_ENC_A,
            ml_encB=ML_ENC_B,
            mr_encA=MR_ENC_A,
            mr_encB=MR_ENC_B
    ):
        self.x = 0.0  # x-position, meters
        self.y = 0.0  # y-position, meters
        self.th = 0.0  # orientation, angle in radians
        
        self.wl = 0.0  # rotational velocity left wheel, rad/s
        self.wr = 0.0  # rotational velocity right wheel, rad/s
        
#         self.wl_prev = 0
#         self.wr_prev = 0
        
        self.wl_smoothed = LowPassFilter(tau)
        self.wr_smoothed = LowPassFilter(tau)
                
        self.dt = dt # time delta in seconds. The control loop runs every dt. Faster means the control loop runs more often. We can increase this to reduce CPU load on our robot.
        self.r = wheel_radius # wheel radius in meters.
        self.l = wheel_sep # wheel separation in meters
        self.max_enc_steps = max_enc_steps # steps in the encoder per revolution
        
        self.real_time = real_time # if True, the wheel velocity measurements consider the actual time elapsed between measurements. If False, it uses the predefined dt value.
        self.last_update = None # last time the control loop ran

        # Pin numbers
        self.motor_L_in1 = ml_in1 # Input 1 (motor left)
        self.motor_L_in2 = ml_in2 # Input 2 (motor left)
        self.motor_R_in3 = mr_in3 # Input 3 (motor right)
        self.motor_R_in4 = mr_in4 # Input 4 (motor right)
        GPIO.setup(self.motor_L_in1, GPIO.OUT)
        GPIO.setup(self.motor_L_in2, GPIO.OUT)
        GPIO.setup(self.motor_R_in3, GPIO.OUT)
        GPIO.setup(self.motor_R_in4, GPIO.OUT)      
        
        # Initialize encoders
        self.ML_ENC = gpiozero.RotaryEncoder(a=ml_encA, b=ml_encB, max_steps=max_enc_steps, wrap=True)
        self.MR_ENC = gpiozero.RotaryEncoder(a=mr_encA, b=mr_encB, max_steps=max_enc_steps, wrap=True)
        self.ml_enc_steps = 0 # left motor encoder value (AKA shaft angle) in steps. Note, this is NOT in radians. It counts from 0 -> MAX_ENC_STEPS -> -MAX_ENC_STEPS (wraps around to -ve value)
        self.mr_enc_steps = 0 # right motor encoder value (AKA shaft angle) in steps. 

        # Initialize motor control pins
        GPIO.setup(ml_pwm, GPIO.OUT)
        GPIO.setup(mr_pwm, GPIO.OUT)
        self.motor_L_pwm = GPIO.PWM(ml_pwm, 1000)  # 1000 Hz frequency. This works well for our motor. Remember that motor speed is controlled by varying the duty cycle of our PWM frequency, and not the frequency itself. 
        self.motor_R_pwm = GPIO.PWM(mr_pwm, 1000)
        self.motor_L_pwm.start(0)
        self.motor_R_pwm.start(0)
    '''
    This method calculates the change in encoder steps between the current and previous time step.
    The logic might seem complicated but this is to deal with the wraparound in value (from 0 -> MAX_ENC_STEPS -> -MAX_ENC_STEPS) that the encoder values have.
    e.g. for the first rotation, the encoder goes from 0 to 3600, then for every rotation thereafter, it goes from -3600 to 3600, then wraps around again.
    '''
    @property
    def pose(self):
        return (self.x, self.y, self.th)
    
    def get_encoder_delta(self, curr_value, prev_value):
        raw_delta = curr_value - prev_value
        delta = raw_delta
        wraparound_thresh = np.ceil((2*self.max_enc_steps + 1) / 2)
        
        if raw_delta > wraparound_thresh:
            delta = raw_delta - (2*self.max_enc_steps+1)
        elif raw_delta < -wraparound_thresh:
            delta = raw_delta + (2*self.max_enc_steps+1)
        
        return delta
            
    def get_dt(self):
        #TODO: Implement smoothing for dt - potentially moving average
        if not self.real_time:
            return self.dt
        
        now = time.time()
        # print(f"Last update: {self.last_update}")
        if self.last_update is None:
            self.last_update = now
            return self.dt
        
        dt = now - self.last_update
        self.last_update = now # this needs to be in between the above and below lines. Don't move it 
        return dt
    
    # Wheel velocities in radians/sec
    '''
    This method reads the encoder values and calculates the wheel velocities in rad/s.
    It uses the get_encoder_delta method to calculate the change in encoder steps between the current and previous time step.
    It then converts this change in steps to radians, and then to radians per second.
    '''
    def read_wheel_velocities(self, dt):
        
        ml_enc_now, mr_enc_now = self.ML_ENC.steps, self.MR_ENC.steps

        # Calculate change in steps, accounting for wrap-around
        ml_enc_delta = self.get_encoder_delta(ml_enc_now, self.ml_enc_steps)
        mr_enc_delta = self.get_encoder_delta(mr_enc_now, self.mr_enc_steps)

        # Convert step change to radians
        ml_delta_rad = ml_enc_delta / self.max_enc_steps * 2 * np.pi 
        mr_delta_rad = mr_enc_delta / self.max_enc_steps * 2 * np.pi

        # Calculate velocities
        self.wl = ml_delta_rad / dt # rad/s
        self.wr = mr_delta_rad / dt # rad/s
        
        # Calculate smoothed velocities
        self.wl_smoothed.update(self.wl, dt)
        self.wr_smoothed.update(self.wr, dt)
        
        # Update previous steps
        self.ml_enc_steps = ml_enc_now
        self.mr_enc_steps = mr_enc_now

        return self.wl, self.wr
  
    '''
    This method sets the motor speed based on the duty cycle provided. 
    It also sets the direction of the motor based on the sign of the duty cycle.
    The duty cycle is the percentage of time the motor is on, and it MUST be a value between -100 and 100.
    '''
    def set_motor_speed(self, left_duty_cycle, right_duty_cycle):      
        # Set direction
        GPIO.output(self.motor_L_in1, GPIO.HIGH if left_duty_cycle >= 0 else GPIO.LOW)
        GPIO.output(self.motor_L_in2, GPIO.LOW if left_duty_cycle >= 0 else GPIO.HIGH)
        GPIO.output(self.motor_R_in3, GPIO.HIGH if right_duty_cycle >= 0 else GPIO.LOW)
        GPIO.output(self.motor_R_in4, GPIO.LOW if right_duty_cycle >= 0 else GPIO.HIGH)
            
        # Set speed
        self.motor_L_pwm.ChangeDutyCycle(abs(left_duty_cycle))
        self.motor_R_pwm.ChangeDutyCycle(abs(right_duty_cycle))


    '''
    This method calculates the linear and angular velocity of the robot based on the wheel velocities.
    It uses the formulae for differential drive robots to calculate the linear and angular velocity.
    This is pretty much identical to Michael's code in the ECE4191 repo.
    '''
    def base_velocity(self, wl, wr):
        v = (wl * self.r + wr * self.r) / 2.0 # linear velocity, m/s, +ve is forward
        w = (wl * self.r - wr * self.r) / self.l # angular velocity, rad/s, +ve is CCW. Note that the negative sign
        # is due to the way the motors are oriented, and so we need it to 'correct' our w calculation
        # so that +ve w is CCW, adhering to convention.
        return v, w
    
    
    """
    This method updates the robot's pose (x, y, theta) based on the wheel velocities.
    It uses the base_velocity method to calculate the linear and angular velocity of the robot.
    It then uses these velocities to update the robot's pose (x, y, theta) based on the kinematic equations.
    """
    def pose_update(self, duty_cycle_ml, duty_cycle_mr):
        # TODO: remove the call to get_dt here, relocate it to the read_wheel_velocities method, and remove the need for it in the pose_update method (below) by having read_wheel_velocities return dt, or the distance travelled (dt*v) directly.
        dt = self.get_dt()
        
        self.set_motor_speed(duty_cycle_ml, duty_cycle_mr)

        wl, wr = self.read_wheel_velocities(dt) # get wheel velocities in rad/s
        v, w = self.base_velocity(wl, wr)
        
        self.x = self.x + dt * v * np.cos(self.th)
        self.y = self.y + dt * v * np.sin(self.th)
        self.th = self.th + w * dt
        
        return self.x, self.y, self.th
    
    
class RobotController:
    def __init__(self, Kp=8, Ki=5, dt=0.1, wheel_radius = 0.028, wheel_sep = 0.222, integral_windup=True, real_time=False):
        self.Kp = Kp            # Proportional gain
        self.Ki = Ki            # Integral gain
        self.dt = dt            # Time step
        
        self.r = wheel_radius
        self.l = wheel_sep
        
        self.min_output = -100  # Minimum duty cycle (-100%)
        self.max_output = 100  # Maximum duty cycle (100%)
        
        self.ml_integral = 0       # Integral term, motor left
        self.mr_integral = 0       # Integral term, motor right
        self.ml_last_update, self.mr_last_update = None, None  # Last update time

        self.anti_integral_windup = integral_windup  # Anti-windup flag
        self.real_time = real_time

    """
    Gets the time of the last measurement/update for the motor ml or mr
    """
    def get_dt(self, motor=None):
        #TODO: Implement smoothing for dt - potentially moving average
        if motor not in ['ml', 'mr']:
            raise ValueError("Motor must be 'ml' or 'mr'.")

        if not self.real_time:
            return self.dt
        
        last_update = f"{motor}_last_update"
        now = time.time()
#         print(f"Last update: {getattr(self, last_update)}")
        if getattr(self, last_update) is None:
            setattr(self, last_update, now)
            return self.dt
        
        dt = now - getattr(self, last_update)
        setattr(self, last_update, now)
        return dt


    def compute(self, w_target, w_actual, motor=None):
        # motor should be either "ml" or "mr" for left or right motor
        if motor not in ['ml', 'mr']:
            raise ValueError("Motor must be 'ml' or 'mr'.")
        integral_attr = f"{motor}_integral"
        last_update = f"{motor}_last_update"
        
        dt = self.get_dt(motor)
        
        error = w_target - w_actual # Calculate the error
        P_out = self.Kp * error # Proportional term
        I_out = self.Ki * getattr(self,integral_attr) # Integral term
        raw_output = P_out + I_out

        if self.anti_integral_windup:
            # Anti-windup - only integrate if output is not saturated
            if self.min_output < raw_output < self.max_output:
                setattr(self, integral_attr, getattr(self, integral_attr) + error * dt)
                # equiv. to self.ml_integral += error * self.dt or self.mr_integral += error * self.dt
        else:
            setattr(self, integral_attr, getattr(self, integral_attr) + error * dt)
                        
            
        return np.clip(raw_output, self.min_output, self.max_output) # Clamp the output to the min/max duty cycle limits
        
    
    def diff_drive_control(self, v_desired, w_desired, wl_actual, wr_actual):
        # v_desired: m/s
        # w_desired (rotation), wl_actual, w_actual (rotation): rad/s
        
        if v_desired == 0 and w_desired == 0:
            return 0, 0, 0, 0
        
        # Calculate desired wheel angular velocities
        wl_desired = (v_desired + self.l*w_desired/2)/self.r
        wr_desired = (v_desired - self.l*w_desired/2)/self.r
        
#         print(f"wl_des (rad/s): {wl_desired:.2f}, wr_des: {wr_desired:.2f}\nwl_des (rps): {wl_desired/(2*np.pi):.2f}, wr_des: {wr_desired/(2*np.pi):.2f}")

        # Compute duty cycles for left and right wheels
        duty_cycle_l = self.compute(wl_desired, wl_actual, 'ml')
        duty_cycle_r = self.compute(wr_desired, wr_actual, 'mr')
        return duty_cycle_l, duty_cycle_r, wl_desired, wr_desired


In [4]:
class TennisBallDetector:
    def __init__(self, lower_color=(20, 100, 100), upper_color=(40, 255, 255), min_radius=10, max_radius=50, min_area=100, line_distance=100):
        self.camera = cv2.VideoCapture(0)  # Adjust if necessary
        self.lower_color = np.array(lower_color)
        self.upper_color = np.array(upper_color)
        self.min_radius = min_radius
        self.max_radius = max_radius
        self.min_area = min_area
        self.balls = []
        self.frame_center = None
        self.last_scanned_time = None
        self.kernel = np.ones((5, 5), np.uint8)
        self.line_distance = line_distance
        self.frame = None
        
        # Define multiple color ranges for different lighting conditions
        self.color_ranges = [
            (np.array([20, 40, 80]), np.array([80, 255, 255])),  # Yellow-green
            (np.array([15, 30, 70]), np.array([85, 255, 255])),  # Broader range
            (np.array([0, 0, 200]), np.array([180, 30, 255]))    # White range
        ]
        
        # Create background subtractor
        self.bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

        if not self.camera.isOpened():
            print("Error: Could not open USB camera.")

    def detect(self, retry=True, max_num_retries=10, retry_interval=1):
        counter = 0
        
        while True:
            ret, self.frame = self.camera.read()
            if not ret:
                print("Error: Failed to capture image.")
                if not retry or counter >= max_num_retries:
                    return False
                counter += 1
#                 time.sleep(retry_interval)
                continue
    
            self.last_scanned_time = time.time()
            self.frame_center = (self.frame.shape[1] // 2, self.frame.shape[0] // 2)
    
            # Apply a slight Gaussian blur to reduce noise
            blurred_frame = cv2.GaussianBlur(self.frame, (5, 5), 0)

            # Convert the frame to the HSV color space
            hsv = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2HSV)

            # Create a mask for the tennis ball color (combine multiple ranges)
            mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
            for lower, upper in self.color_ranges:
                mask |= cv2.inRange(hsv, lower, upper)

            # Apply background subtraction
            fg_mask = self.bg_subtractor.apply(self.frame)
            
            # Combine color mask with foreground mask
            combined_mask = cv2.bitwise_and(mask, fg_mask)

            # Perform morphological operations to remove noise and improve detection
            combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, self.kernel)
            combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, self.kernel)

            # Find contours
            contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            self.balls = []

            for contour in contours:
                area = cv2.contourArea(contour)
                if area > self.min_area:
                    perimeter = cv2.arcLength(contour, True)
                    circularity = 4 * np.pi * (area / (perimeter ** 2))
                    if circularity > 0.7:  # Enforce circularity check
                        (x, y), radius = cv2.minEnclosingCircle(contour)
                        if self.min_radius < radius < self.max_radius:
                            self.balls.append(((int(x), int(y)), int(radius)))
            
            if self.balls:
                return True
            
            if not retry or counter >= max_num_retries:
                return False
            
            counter += 1
#             time.sleep(retry_interval)

    def get_ball_count(self):
        return len(self.balls)

    def get_ball_locations(self):
        return [ball[0] for ball in self.balls]

    def get_ball_distance_from_center(self, ball_index):
        if 0 <= ball_index < len(self.balls):
            ball_center = self.balls[ball_index][0]
            dx = ball_center[0] - self.frame_center[0]
            dy = ball_center[1] - self.frame_center[1]
            distance_pixels = np.sqrt(dx**2 + dy**2)
            angle_radians = np.arctan2(dy, dx)
            return distance_pixels, angle_radians
        else:
            return None

    def get_ball_vertical_y(self, ball_index=0):
        if 0 <= ball_index < len(self.balls):
            return self.balls[ball_index][0][1]
        else:
            return None
        
    def draw_balls(self, frame=None):
        if frame is None:
            frame = self.frame
        for i, (center, radius) in enumerate(self.balls):
            color = (0, 255, 0) if i == 0 else (0, 255, 255)  # Green for largest, Yellow for others
            cv2.circle(frame, center, radius, color, 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
        return frame

    def get_last_scanned_time(self):
        return self.last_scanned_time

    def draw_vertical_lines(self, frame=None):
        if frame is None:
            frame = self.frame
        height, width = frame.shape[:2]
        left_x = width // 2 - self.line_distance // 2
        right_x = width // 2 + self.line_distance // 2
        
        cv2.line(frame, (left_x, 0), (left_x, height), (255, 0, 0), 2)
        cv2.line(frame, (right_x, 0), (right_x, height), (255, 0, 0), 2)
        
        return frame

    def is_ball_between_lines(self, ball_index=0):
        if 0 <= ball_index < len(self.balls):
            ball_center, ball_radius = self.balls[ball_index]
            height, width = self.camera.get(cv2.CAP_PROP_FRAME_HEIGHT), self.camera.get(cv2.CAP_PROP_FRAME_WIDTH)
            left_x = width // 2 - self.line_distance // 2
            right_x = width // 2 + self.line_distance // 2
            
            return (left_x <= ball_center[0] - ball_radius) and (ball_center[0] + ball_radius <= right_x)
        else:
            return False

    def get_ball_distance_from_lines(self, ball_index=0):
        if 0 <= ball_index < len(self.balls):
            ball_center, _ = self.balls[ball_index]
            height, width = self.camera.get(cv2.CAP_PROP_FRAME_HEIGHT), self.camera.get(cv2.CAP_PROP_FRAME_WIDTH)
            left_x = width // 2 - self.line_distance // 2
            right_x = width // 2 + self.line_distance // 2
            
            if ball_center[0] < left_x:
                return -(left_x - ball_center[0])
            elif ball_center[0] > right_x:
                return ball_center[0] - right_x
            else:
                return 0  # Ball is between the lines
        else:
            return None

    def set_line_distance(self, distance):
        self.line_distance = distance

    def display_frame(self):
        if self.frame is not None:
            frame_with_balls = self.draw_balls()
            frame_with_lines = self.draw_vertical_lines(frame_with_balls)
            frame_rgb = cv2.cvtColor(frame_with_lines, cv2.COLOR_BGR2RGB)
            display.clear_output(wait=True)
            plt.imshow(frame_rgb)
            plt.show()

In [5]:
# PASTE YOUR TENTACLE PLANNER HERE

class TentaclePlanner:
    def __init__(
        self,
        obstacles  = [],
        dt = 0.2,
        steps = 15, # Number of steps the planner uses when simulating a path #CHANGE
        alpha = 1, # Weighs the positional error in the cost function. Higher value prioritises goal positions.
        beta = 0.1, # Weighs the rotational error of the cost function. higher value prioritises correct final orientation.
        max_linear_velocity = 0.23, # 0.23 m/s is at about 75% duty cycle.
        max_angular_velocity = 2.1, # 2.1 rad/s is about 78% duty cycle
        max_linear_tolerance = 0.1, # meters
        max_angular_tolerance = 0.1, # radians, 0.15 rad = 8.6 deg, 0.08 = 4.6 deg
        max_acceleration = 0.3,
        max_angular_acceleration=0.5,
        current_linear_velocity = 0.0,
        current_angular_velocity = 0.0
       
    ):
        self.dt = dt
        self.steps = steps
        self.alpha = alpha
        self.beta = beta
        self.obstacles = np.array(obstacles)
        self.max_linear_velocity = max_linear_velocity
        self.max_angular_velocity = max_angular_velocity
        self.max_linear_tolerance = max_linear_tolerance
        self.max_angular_tolerance = max_angular_tolerance 
        self.max_acceleration = max_acceleration
        self.max_angular_acceleration = max_angular_acceleration
        self.current_linear_velocity = current_linear_velocity
        self.current_angular_velocity = current_angular_velocity
        
        self.tentacles = self._generate_tentacles()

#     def _generate_tentacles(self):
#         linear_velocities = np.linspace(-self.max_angular_velocity, self.max_linear_velocity, 30) #CHANGE
#         angular_velocities = np.linspace(-self.max_angular_velocity, self.max_angular_velocity,30)
#         tentacles = [(v, w) for v in linear_velocities for w in angular_velocities]
#         pure_linear_tentacles = [(v, 0.0) for v in linear_velocities if v != 0] # pure linear motion options
#         tentacles.extend(pure_linear_tentacles)
#         pure_angular_tentacles = [(0.0, w) for w in angular_velocities if w != 0]
#         tentacles.extend(pure_angular_tentacles)
#         tentacles.append((0.0, 0.0))  # Add stop option
#         tentacles = list(set(tentacles))

#         return tentacles

    def _generate_tentacles(self):
        # Increased resolution for velocities
        linear_velocities = np.linspace(-self.max_linear_velocity, self.max_linear_velocity, 30)
        angular_velocities = np.linspace(-self.max_angular_velocity, self.max_angular_velocity, 30)

        # Basic tentacles with more density
        tentacles = [(v, w) for v in linear_velocities for w in angular_velocities]

        # Fine granularity near zero for more precise control
        fine_linear = np.linspace(-0.1 * self.max_linear_velocity, 0.1 * self.max_linear_velocity, 5)
        fine_angular = np.linspace(-0.1 * self.max_angular_velocity, 0.1 * self.max_angular_velocity, 5)
        tentacles.extend([(v, w) for v in fine_linear for w in fine_angular])

        # Pure linear and pure rotational motions
        tentacles.extend([(v, 0) for v in linear_velocities if v != 0])
        tentacles.extend([(0, w) for w in angular_velocities if w != 0])

        # Random perturbations for exploration
        random_perturbations = [(v + np.random.uniform(-0.05, 0.05), w + np.random.uniform(-0.05, 0.05))
                                for v, w in tentacles]
        tentacles.extend(random_perturbations)

        # Remove duplicates and return
        return list(set(tentacles))
    
    
    
    def is_goal_reached(self, goal_x, goal_y, goal_th, x, y, th):
        distance_to_goal = np.hypot(goal_x - x, goal_y - y)
        angular_error = np.arctan2(np.sin(goal_th - th), np.cos(goal_th - th))
        return distance_to_goal <= self.max_linear_tolerance #and abs(angular_error) <= self.max_angular_tolerance

    def trapezoidal_profile(self, distance, current_velocity, max_velocity, max_acceleration):
        acceleration_distance = (max_velocity ** 2) / (2 * max_acceleration)
        if distance == 0:
            velocity = current_velocity + max_acceleration * self.dt
        elif 0 < distance <= 2 * acceleration_distance:
            t_accel = np.sqrt(distance / max_acceleration)
            velocity = min(current_velocity + max_acceleration * t_accel, max_velocity)
        else:
            velocity = max_velocity
        return velocity

    def _roll_out(self, v, w, goal_x, goal_y, goal_th, x, y, th):
        for _ in range(self.steps):
            x += self.dt * v * np.cos(th)
            y += self.dt * v * np.sin(th)
            th += w * self.dt

            if self._check_collision(x, y):
                return np.inf
            
        dist2goal = np.hypot(goal_x - x, goal_y - y)
        
        e_th = np.arctan2(np.sin(goal_th - th), np.cos(goal_th - th))
        
        cost = self.alpha * dist2goal + self.beta * abs(e_th)

        return cost

    def _check_collision(self, x, y):
        if len(self.obstacles) == 0:
            return False
        min_dist = np.min(np.hypot(x - self.obstacles[:, 0], y - self.obstacles[:, 1]))
        return min_dist < 0.1

    def _angle_between_points(self, p1, p2):
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        return np.arctan2(dy, dx)
    
    
    def _normalise_angle(self, angle):  # in radians
        return (angle + np.pi) % (2 * np.pi) - np.pi
    
    def _shortest_angular_distance(self, from_angle, to_angle):
        """
        Calculate the shortest angular distance between two angles.
        """
        diff = self._normalise_angle(to_angle - from_angle)
        if diff > np.pi:
            diff -= 2 * np.pi
        return diff
    

        
#     def _plan_tentacles(self, goal_x, goal_y, goal_th, x, y, th, use_trapezoidal=True):
#         if self.is_goal_reached(goal_x, goal_y, goal_th, x, y, th):
# #             print("Goal is within tolerance, stopping planner function")
#             return (0.0, 0.0)

#         costs = [self._roll_out(v, w, goal_x, goal_y, goal_th, x, y, th) for v, w in self.tentacles]
#         best_idx = np.argmin(costs)
#         v = self.tentacles[best_idx][0]
#         w = self.tentacles[best_idx][1]
        
#         dx, dy = goal_x - x, goal_y - y
#         distance = np.hypot(dx, dy)
#         angle_to_goal = self._angle_between_points((x, y), (goal_x, goal_y))
        
#         # Calculate the shortest angular distance -pi to pi
#         angle_diff = self._shortest_angular_distance(th, angle_to_goal)
        
# #         if use_trapezoidal:
# #              # Calculate linear and angular velocities
# #             v = self.trapezoidal_profile(distance, self.current_linear_velocity, self.max_linear_velocity, self.max_acceleration)
# #             w = self.trapezoidal_profile(angle_diff, self.current_angular_velocity, self.max_angular_velocity, self.max_angular_acceleration)
        
#         return v, w

    def _plan_tentacles(self, goal_x, goal_y, goal_th, x, y, th):
        
#         current_goal = goals[current_goal_index]
#         goal_x, goal_y, goal_th = current_goal

#         if self.is_goal_reached(goal_x, goal_y, goal_th, x, y, th):
#             # Move to the next goal if available
#             if current_goal_index < len(goals) - 1:
#                 current_goal_index += 1
#                 print(f"Reached goal {current_goal_index-1}, moving to goal {current_goal_index}")
#                 current_goal = goals[current_goal_index]
#                 goal_x, goal_y, goal_th = current_goal
#             else:
#             return (0.0, 0.0)  # No more goals to process
        
        if self.is_goal_reached(goal_x, goal_y, goal_th, x, y, th):
            return (0.0, 0.0)
        
        
        tentacle_num = 15
        angle_to_goal = self._angle_between_points((x, y), (goal_x, goal_y))
        angle_diff = self._shortest_angular_distance(th, angle_to_goal)

        # Define tentacles for turning
        turning_tentacles = [(0, w) for w in np.linspace(-self.max_angular_velocity, self.max_angular_velocity, tentacle_num) if w != 0]
        best_turn_tentacle = min(turning_tentacles, key=lambda t: abs(angle_diff - t[1]))
        best_turn_rate = best_turn_tentacle[1]

        if abs(angle_diff) > self.max_angular_tolerance:
            return (0.0, best_turn_rate)

        # If aligned, use linear tentacles
        linear_tentacles = [(v, 0) for v in np.linspace(0, self.max_linear_velocity, tentacle_num)]
        best_linear_tentacle = min(linear_tentacles, key=lambda t: self._roll_out(t[0], 0, goal_x, goal_y, goal_th, x, y, th))

        if abs(angle_diff) <= self.max_angular_tolerance:
            return (best_linear_tentacle[0], 0.0)

        return (best_linear_tentacle[0], best_turn_rate)

    
    
    def _rotate(self, goal_x, goal_y, goal_th, x, y, th):        
        # Calculate the shortest angular distance
        
        value = 0.5
        
        diff = goal_th - th
        angle_diff = (diff + np.pi) % (2 * np.pi) - np.pi
    
        if abs(angle_diff) > 0.05:
            w = -abs(value) if angle_diff < 0 else abs(value)            
            return 0.0, w
                
        else:
            return 0.0, 0.0

    def get_control_inputs(self, goal_x, goal_y, goal_th, x, y, th, strategy='tentacles'):
        if strategy == 'rotate':
            v, w = self._rotate(goal_x, goal_y, goal_th, x, y, th)
        elif strategy == 'tentacles':
            v, w = self._plan_tentacles(goal_x, goal_y, goal_th, x, y, th)
        else:
            raise ValueError("Invalid strategy. Must be 'rotate_then_drive' or 'tentacles' or 'smooth_pursuit'.")
        
        self.current_linear_velocity = v
        self.current_angular_velocity = w
        return {
            'linear_velocity': v,
            'angular_velocity': w,
            'current_x': x,
            'current_y': y,
            'current_theta': th,
            'goal_x': goal_x,
            'goal_y': goal_y,
            'goal_theta': goal_th
        }

    def visualize_rotate_then_drive(self, goal_x, goal_y, goal_th, x, y, th, steps=100):
        plt.figure(figsize=(10, 10))
        plt.scatter(x, y, color='green', s=100, label='Start')
        plt.scatter(goal_x, goal_y, color='blue', s=100, label='Goal')

        x_traj, y_traj = [x], [y]
        th_current = th

        for _ in range(steps):
            v, w = self._rotate_then_drive(goal_x, goal_y, goal_th, x_traj[-1], y_traj[-1], th_current)
            th_current += w * self.dt
            x_new = x_traj[-1] + v * np.cos(th_current) * self.dt
            y_new = y_traj[-1] + v * np.sin(th_current) * self.dt
            x_traj.append(x_new)
            y_traj.append(y_new)

            if self.is_goal_reached(goal_x, goal_y, goal_th, x_new, y_new, th_current):
                break

        plt.plot(x_traj, y_traj, 'g-', linewidth=2, label='Path')
        plt.legend()
        plt.title('Rotate-then-Drive Path')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.grid(True)
        plt.axis('equal')
        plt.show()

    def visualize_tentacles(self, x, y, th, extrapolation_steps=100):
        plt.figure(figsize=(12, 12))
        plt.scatter(x, y, color='green', s=100, label='Start')

        colors = plt.cm.rainbow(np.linspace(0, 1, len(self.tentacles)))

        for (v, w), color in zip(self.tentacles, colors):
            x_traj, y_traj = [x], [y]
            th_traj = th
            for _ in range(self.steps):
                x_new = x_traj[-1] + self.dt * v * np.cos(th_traj) * extrapolation_steps
                y_new = y_traj[-1] + self.dt * v * np.sin(th_traj) * extrapolation_steps
                x_traj.append(x_new)
                y_traj.append(y_new)
                th_traj += w * self.dt

                if _ == self.steps // 2:
                    dx, dy = x_new - x_traj[-2], y_new - y_traj[-2]
                    arrow = Arrow(x_traj[-2], y_traj[-2], dx, dy, width=0.2, color=color)
                    plt.gca().add_patch(arrow)

            plt.plot(x_traj, y_traj, '-', color=color, linewidth=2, label=f'v={v:.2f}, w={w:.2f}')

        if len(self.obstacles) > 0:
            plt.scatter(self.obstacles[:, 0], self.obstacles[:, 1], color='red', s=50, label='Obstacles')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.title('Tentacle Visualization')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.grid(True)
        plt.axis('equal')
        plt.tight_layout()
        plt.show()

    def visualize_plan(self, goal_x, goal_y, goal_th, x, y, th, use_straight_line=False, extrapolation_steps=100):
        v, w = self.plan(goal_x, goal_y, goal_th, x, y, th, use_straight_line)

        plt.figure(figsize=(10, 10))
        plt.scatter(x, y, color='green', s=100, label='Start')
        plt.scatter(goal_x, goal_y, color='blue', s=100, label='Goal')

        x_traj, y_traj = [x], [y]
        th_traj = th
        for _ in range(self.steps):
            x_traj.append(x_traj[-1] + self.dt * v * np.cos(th_traj) * extrapolation_steps)
            y_traj.append(y_traj[-1] + self.dt * v * np.sin(th_traj) * extrapolation_steps)
            th_traj += w * self.dt
        plt.plot(x_traj, y_traj, 'g-', linewidth=2, label='Chosen Path')

        if len(self.obstacles) > 0:
            plt.scatter(self.obstacles[:, 0], self.obstacles[:, 1], color='red', s=50, label='Obstacles')
        plt.legend()
        plt.title(f'Planned Path (v={v:.2f}, w={w:.2f})')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.grid(True)
        plt.axis('equal')
        plt.show()



# # Create an instance of the planner
# obstacles = np.array([[1, 1], [2, 2], [3, 1]])  # Example obstacles
# planner = TentaclePlanner(obstacles, max_linear_velocity=1.0, max_angular_velocity=0.5)

# # Get control inputs for tentacle-based planning
# inputs = planner.get_control_inputs(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0)
# print("Tentacle-based planning:", inputs)

# # Get control inputs for straight-line planning
# inputs_straight = planner.get_control_inputs(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0, use_straight_line=True)
# print("Straight-line planning:", inputs_straight)

# # Visualize the plan
# planner.visualize_plan(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0)
# planner.visualize_plan(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0, use_straight_line=True)

In [6]:
# class LogData:
#     def __init__(self):
#         self.poses = []
#         self.velocities = []
#         self.desired_velocities = []
#         self.duty_cycle_commands = []
#         self.error_sums = []
#         self.errors = []
#         self.actual_dts = []

class Orchestrator:
    def __init__(self):
        self.running = True
        self.last_update = None
        self.dt = 0.1
        
        self.robot = None
        self.controller = None
        self.detector = None
        self.planner = None
        self.control_thread = None

        self.poses = []
        self.goal_positions = []
        self.velocities = []
        self.desired_velocities = []
        self.duty_cycle_commands = []
        self.error_sums = []
        self.errors = []
        self.actual_dts = []
        self.scan_locations = []
        
        self.duration = 0
        self.start_time = 0

                
    def get_dt(self): 
        now = time.time()
        if self.last_update is None:
            self.last_update = now
            return self.dt
        
        dt = now - self.last_update
        self.last_update = now # this needs to be in between the above and below lines. Don't move it 
        return dt
    
    
    """
    Notes to myself (Patrick): 
    - Outer tennis court quadrant is 5.48m deep, 4.12m wide. Inner quadrants are 6.40m deep.
    - 
    """
    def control_loop(self, width=4.12, depth=5.48):
        try:
            # Log data
            x, y, th = self.robot.pose
            self.poses.append([x, y, th])
            self.velocities.append([self.robot.wl, self.robot.wr])

            # Calculate number of vertical scan lines based on maximum camera detection distance
            max_scan_distance = .8
            num_scan_lines = int(np.ceil((width-(2*max_scan_distance)) / (2*max_scan_distance)))
            scan_lines = np.linspace(max_scan_distance, width-max_scan_distance, num_scan_lines)
            scan_lines = [-v for v in scan_lines]
            
            # Calculate vertical stop points
            num_scan_points = int(np.ceil((depth-(2*max_scan_distance)) / (2 * max_scan_distance)))
            scan_points = np.linspace(max_scan_distance, depth-max_scan_distance, num_scan_points)
            
            scan_line_idx = 0
            scan_point_idx = 0

            # Define scan directions
            UP = 1
            DOWN = -1
            current_direction = UP
            while self.running:

                scan_direction = "anticlockwise" if current_direction == UP else "clockwise"
                scan_line_distance = scan_lines[scan_line_idx]
                scan_point_distance = scan_points[scan_point_idx]

                # Move to the next scanning position
                print(f"Moving to next scanning position: col: {scan_line_idx}, row: {scan_point_idx}")
                self.move_to_scan_position(scan_point_distance, scan_line_distance, current_direction)

                # Scan for ball
                ball_found = self.scan_for_ball(scan_direction=scan_direction, simulate_camera=False)
                if ball_found:
                    self.approach_ball(50)
                    print("Going home now.")
                    self.navigate(0, 0, self.robot.th) # go home
                    return  # End the control loop

                # Move to the next scan point or line
                scan_point_idx, scan_line_idx, current_direction = self.update_scan_indices(scan_point_idx, scan_line_idx, scan_lines, scan_points, current_direction)

                if scan_line_idx >= len(scan_lines):
                    # This doesn't actually run because self.update_scan_indices doesn't let scan_line_idx > len()
                    print("Scanning complete")
                    self.navigate(0, 0, self.robot.th) # return to origin
                    break

        except Exception as e:
            print(f"\nUnhandled Exception in Orchestrator control loop: {e}. Stopping Robot.\n")
            self.stop()

            
            
    def move_to_scan_position(self, scan_point_distance, scan_line_distance, direction):
            
        """# Proper way to move to our next scan location"""
        try:
            goal_angle = 0 if direction == 1 else -np.pi
            self.navigate(scan_point_distance, scan_line_distance, goal_angle)
            self.rotate(goal_angle, absolute=True)

        except Exception:
            raise
        
        
    def navigate(self, goal_x, goal_y, goal_th, strategy="tentacles"):
        try:
            while True and self.running:
                inputs = self.planner.get_control_inputs(goal_x, goal_y, goal_th, *self.robot.pose, strategy=strategy)
                self.apply_control_inputs(inputs)
                self.goal_positions.append([goal_x, goal_y, goal_th])
                if inputs['linear_velocity'] == 0.0 and inputs['angular_velocity'] == 0.0:
                    # print("Rotation complete: planner indicates no more movement needed.")
                    break

                time.sleep(self.dt)
              
        except KeyboardInterrupt:
            print("Keyboard interrupt caught inside self.navigate() - reraising.")
            raise
            
        except Exception:
            print("Unhandled exception inside self.navigate()")
            raise Exception

        return

        
    def update_scan_indices(self, scan_point_idx, scan_line_idx, scan_lines, scan_points, current_direction):
        UP, DOWN = 1, -1
        
        scan_point_idx += current_direction # Increment scan_point_idx according to the current direction
        
        if scan_point_idx < 0 or scan_point_idx >= len(scan_points): # Check if we've reached the end of the vertical scan
            
            if current_direction == UP: # We've completed a vertical scan, move to the next scan line
                # We were moving up, so now we need to start moving down
                # and move to the next scan line to the left
                current_direction = DOWN
                scan_line_idx += 1
                scan_point_idx = len(scan_points) - 1  # Start from the top
            else:  # current_direction == DOWN
                # We were moving down, so now we need to start moving up
                # and move to the next scan line to the left
                current_direction = UP
                scan_line_idx += 1
                scan_point_idx = 0  # Start from the bottom
            
            # Check if we've completed all scan lines
            if scan_line_idx >= len(scan_lines):
                # We've completed all scan lines, reset to start a new sweep if needed
                scan_line_idx = 0
                current_direction = UP
                scan_point_idx = 0
        
        return scan_point_idx, scan_line_idx, current_direction
            
        
            
    """
    This is a subroutine that rotates on the spot and scans for the ball. 
    """        
    def scan_for_ball(self, 
                      scan_direction='clockwise', 
                      simulate_camera=False,
                      rotation_thres=6.27, # just below 360 degrees
                      rotation_increment = 0.5236 # 30 * np.pi/180  # 30 degrees
                     ):
        
        rotation_angle = 0
        initial_theta = self.robot.th
        self.scan_locations.append([*self.robot.pose]) # log data
        print("\nBeginning rotational scan now.")
        while rotation_angle <= rotation_thres and self.running:
            if simulate_camera:
                print("Simulating camera wait...")
                time.sleep(2)  # Wait for 2 seconds to simulate camera processing
                # Simulate ball detection (randomly for demonstration)

#                 ball_detected = random.choice([True, False])
                ball_detected = False
                is_between = True

                is_between = random.choice([True, False]) if ball_detected else False
                distance = random.uniform(-1, 1) if ball_detected else 0
            else:
                # Take an image and detect tennis balls
                ball_detected = self.detector.detect()
#                 self.detector.display_frame()
                distance = self.detector.get_ball_distance_from_lines()
                ball_y_dist = self.detector.get_ball_vertical_y()
                time.sleep(1)
            if ball_detected:
                if distance == 0:
                    print("Ball detected between lines. Exiting scan_for_ball(). Returning True")
                    return ball_y_dist
                else:
                    print(f"Ball detected. Distance from center: {distance:.2f}")
                    rotation_direction = 1 if (distance < 0) else -1
                    scaling_factor = 0.02
                    rotation_adjustment = rotation_direction * 10 * np.pi/180
                    print(f"Rotating by {rotation_adjustment} radians to adjust.\n")
                    self.rotate(rotation_adjustment)  # Rotate by 10 degrees
                    return True
            else:
                rotation_angle += rotation_increment
                if rotation_angle > rotation_thres:
                    break
                else:
                    print(f"No ball detected. Rotating by {rotation_increment * 180/np.pi:.2f} degrees.")
                    self.rotate(rotation_increment if scan_direction == 'clockwise' else -rotation_increment)
                    

        # If we've rotated full threshold and haven't found a ball, rotate back to initial position
        print("No ball found. Exiting scan routine.")
#         self.rotate(initial_theta - self.robot.th) # We can REMOVE THIS if using proper motion planning
        return False

    """
    approach ball()
    """

    def approach_ball(self, ball_y=30, y_threshold=30):
        print("Attempting to approach ball now. ")
        max_iterations = 10  # Prevent infinite loop
        iterations = 0
        
        while iterations < max_iterations and self.running:
            # Calculate distance to move
            # Combine linear and exponential scaling
            linear_factor = 0.001  # Adjust this value to change the linear component
            exp_factor = 0.0  # Adjust this value to change the exponential component
            # Take a new picture
            
            # Get the ball's vertical position
            
            if ball_y is None:
                
                print("Ball not detected. Cannot approach.")
                return False
            
            if ball_y < y_threshold:
                print(f"Ball is close enough (y = {ball_y}). Approach successful.")
                return True
            
            distance = linear_factor * ball_y + exp_factor * np.exp(ball_y / 50)

            
            # Limit the maximum movement to prevent overshooting
            max_movement = 0.5  # meters
            distance = min(distance, max_movement)
            
            print(f"Moving forward {distance:.2f} meters (ball_y = {ball_y})")
            new_x = self.robot.x + distance * np.cos(self.robot.th)
            new_y = self.robot.y + distance * np.sin(self.robot.th)
                
            self.navigate(new_x, new_y, self.robot.th)
            
            self.detector.detect()
            ball_y = self.detector.get_ball_vertical_y(0)


            iterations += 1
        
        print(f"Could not approach ball within {max_iterations} iterations.")
        
        return False
            
    
    """
    EMERGENCY FIX methods are methods that we would ideally not need, but are required as workarounds
    due to the limited implementation and software development human resources that we have.
    - Ideally, our motion planner would be good enough to accept x, y, th coords without having funky outputs
    - Additionally, the motion planner (e.g. TentaclePlanner) would ideally have a good enough interface that we wouldn't
    need to have yet another abstraction of an abstraction
    """
    
    """
    EMERGENCY FIX METHOD FOR MILESTONE 1
    Rotates the robot. 
    - Absolute set False means that the robot rotates by the angle parameter. 
    - Absolute set True means that the robot will rotate to the angle supplied. Actual rotation may or may not be angle.
    
    """
    def rotate(self, angle, absolute=False):
        print("Rotate method entered.")
        goal_theta = angle
        if not absolute:
            goal_theta += self.robot.th
        while True and self.running:
            inputs = self.planner.get_control_inputs(
                self.robot.x, self.robot.y, goal_theta, 
                self.robot.x, self.robot.y, self.robot.th, 
                strategy="rotate"
            )
            self.apply_control_inputs(inputs)
            if inputs['linear_velocity'] == 0.0 and inputs['angular_velocity'] == 0.0:
#                 print("Rotation complete: planner indicates no more movement needed.")
                break
            time.sleep(self.dt)
        return 

    """ 
    Applies control inputs from the Motion Planner (e.g. TentaclePlanner) into the robot.
    """
    def apply_control_inputs(self, inputs):
        if len(inputs) == 2:
            linear = inputs[0]
            angular = inputs[1]
        else:
            linear = inputs['linear_velocity']
            angular = inputs['angular_velocity']
        
        duty_cycle_l, duty_cycle_r, wl_desired, wr_desired = self.controller.drive(
            linear, 
            angular, 
            self.robot.wl_smoothed.value, 
            self.robot.wr_smoothed.value
        )
        # Log data
        x, y, th = self.robot.pose
        self.poses.append([x, y, th])
        self.velocities.append([self.robot.wl, self.robot.wr])
        self.duty_cycle_commands.append([duty_cycle_l, duty_cycle_r])
        self.desired_velocities.append([wl_desired, wr_desired])
        
        self.robot.pose_update(duty_cycle_l, duty_cycle_r)
        
    """
    Start the Orchestrator, initialise children class instances, and begin operation.
    """
    def start(self):
        if self.robot is None:
            print("Initialising robot.")
            self.robot = DiffDriveRobot(0.05, real_time=True)

        if self.controller is None:
            print("Initialising controller.")
            self.controller = RobotController(real_time=True)
            
        if self.detector is None:
            print("Initialising tennis ball detector.")
            self.detector = TennisBallDetector()
            
        if self.planner is None:
            print("Initialising tentacle planner.")
            self.planner = TentaclePlanner(max_linear_velocity=0.2, max_angular_velocity=2)
        """
        Multi-threaded Approach (comment this or the one below out)
        """
        
        self.control_thread = threading.Thread(target=self.control_loop) # you can change which control loop is called. Supply the function name, no parentheses.
        self.start_time = time.time()
        self.control_thread.start()
        
    """
    Stops the Orchestrator and stops the robot.
    """
    def stop(self):
        self.running = False
        self.control_thread.join() 
        self.robot.set_motor_speed(0, 0)
        self.duration = time.time() - self.start_time
        print(f"Ended run. Robot ran for: {self.duration:.2f} s")

In [7]:
def update_plot(orchestrator, fig, axes):
    axes_flat = axes.flatten()  # Flatten the 2D array of axes
    for ax in axes_flat:
        ax.clear()
    
    # Plot 1: Robot path and orientation
    poses = np.array(orchestrator.poses)
    if len(poses) > 0:
        axes[0, 0].plot(np.array(poses)[:,0], np.array(poses)[:,1])
        x, y, th = poses[-1]
        axes[0, 0].plot(x, y, 'k', marker='+')
        axes[0, 0].quiver(x, y, 0.1*np.cos(th), 0.1*np.sin(th))
    axes[0, 0].set_xlabel('x-position (m)')
    axes[0, 0].set_ylabel('y-position (m)')
    axes[0, 0].set_title(f"Robot Pose Over Time. Kp: {orchestrator.controller.Kp}, Ki: {orchestrator.controller.Ki}")
    axes[0, 0].axis('equal')
    axes[0, 0].grid()

    # Plot 2: Duty cycle commands
    duty_cycle_commands = np.array(orchestrator.duty_cycle_commands)
    if len(duty_cycle_commands) > 0:
        duty_cycle_commands = np.array(duty_cycle_commands)
        axes[0, 1].plot(duty_cycle_commands[:, 0], label='Left Wheel')
        axes[0, 1].plot(duty_cycle_commands[:, 1], label='Right Wheel')
    
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Duty Cycle')
    axes[0, 1].set_title('Duty Cycle Commands Over Time')
    axes[0, 1].legend()
    axes[0, 1].grid()
    
    # Plot 3: Wheel velocities
    velocities = np.array(orchestrator.velocities)
    desired_velocities = np.array(orchestrator.desired_velocities)
    if len(velocities) > 0 and len(desired_velocities) > 0:
        axes[1, 0].plot(velocities[:, 0], label='Left Wheel')
        axes[1, 0].plot(velocities[:, 1], label='Right Wheel')
        axes[1, 0].plot(desired_velocities[:, 0], label='Desired Left Wheel')
        axes[1, 0].plot(desired_velocities[:, 1], label='Desired Right Wheel')
    axes[1, 0].set_xlabel('Time Step')
    axes[1, 0].set_ylabel('Wheel Velocity (rad/s)')
    axes[1, 0].set_title('Wheel Velocity vs. Time')
    axes[1, 0].legend()
    axes[1, 0].grid()
    
    # Plot 4: Goal Positions vs. actual position
    goal_positions = np.array(orchestrator.goal_positions)
    poses = np.array(orchestrator.poses)

    # Add (0, 0) to both goal_positions and poses
    goal_positions = np.vstack(((0, 0, 0), goal_positions))
    poses = np.vstack(([0, 0, 0], poses))
    scan_locations = np.array(orchestrator.scan_locations)

    axes[1, 1].plot(0, 0, 'ko', markersize=10, label='Start (0, 0)')  # Add point at (0, 0)

    if len(poses) > 0: 
        axes[1, 1].plot(poses[:, 0], poses[:, 1], 'b-', label='Actual Path')
        axes[1, 1].scatter(poses[:, 0], poses[:, 1], color='b', s=5)  # Add dots for each position with custom size
    if len(goal_positions) > 1:
        axes[1, 1].plot(goal_positions[:, 0], goal_positions[:, 1], 'r--', label='Goal Path')
        axes[1, 1].plot(goal_positions[:, 0], goal_positions[:, 1], 'r.')  # Add dots for each goal position
    if len(scan_locations) > 1:
        axes[1, 1].scatter(scan_locations[:, 0], scan_locations[:, 1], color='g', s=20, label='Scan Locations')   # Add dots for each scan position

    axes[1, 1].set_xlabel('x-position (m)')
    axes[1, 1].set_ylabel('y-position (m)')
    axes[1, 1].set_title(f"Robot Positions. {orchestrator.duration:.2f} sec run.")
    axes[1, 1].axis('equal')
    axes[1, 1].grid(True)
    axes[1, 1].legend()
    
    fig.tight_layout()
    fig.canvas.draw()
#     display.clear_output(wait=True)
    display.display(fig)


In [8]:
orchestrator = Orchestrator()


In [ ]:
orchestrator.start()


# Set up the plot
%matplotlib inline
plt.ion()  # Turn on interactive mode

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Main loop for updating the plot
try:
    while True:
        pass
#         update_plot(orchestrator, fig, axes)
#         time.sleep(3)  # Update plot every 3 seconds
except KeyboardInterrupt:
    print("Stopping the simulation...")
finally:
    orchestrator.stop()
    update_plot(orchestrator, fig, axes)
    plt.close(fig)


Initialising robot.
Initialising controller.
Initialising tennis ball detector.
Initialising tentacle planner.
Moving to next scanning position: col: 0, row: 0
Rotate method entered.

Beginning rotational scan now.
Ball detected. Distance from center: -102.00
Rotating by 0.17453292519943295 radians to adjust.

Rotate method entered.
Attempting to approach ball now. 
Moving forward 0.05 meters (ball_y = 50)
Ball not detected. Cannot approach.
Going home now.
Stopping the simulation...
Ended run. Robot ran for: 100.37 s


In [ ]:
# To regenerat e the plot in a nice way, run this cell:
%matplotlib inline
plt.ion()  # Turn on interactive mode

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
update_plot(orchestrator, fig, axes)

In [ ]:
robot.set_motor_speed(0, 0)

In [ ]:

GPIO.setwarnings(False)
robot = DiffDriveRobot(0.05, real_time=True)

In [ ]:
# Here are some example position commands we can put in.

#goal = (0.5, 0.5, 0)
#goal = (2, 2, np.pi/2)
goal = (1, 1, 0)
# goal = (0, 0, 60/180 * np.pi)




# Initialising controller, logging variables 
# Controller Parameters
controller = RobotController(real_time=True) # using default Kp = 8, Ki=9
planner = TentaclePlanner() # remember to paste in the class in the cell where I told you


robot.x = 0
robot.y = 0
robot.th = 0
robot.wl = 0
robot.wr = 0

poses = []
goal_positions = []
velocities = []
desired_velocities = []
duty_cycle_commands = []
error_sums = []
errors = []
actual_dts = []

count = 0


try:
    goal_positions.append([0, 0, 0])
    goal_positions.append([goal[0],goal[1],goal[2]])

    start_time = time.time()
    while True: 
        
        ball_found = self.scan_for_ball(scan_direction=scan_direction, simulate_camera=False)
        if ball_found:
            self.approach_ball()
            self.navigate(0, 0, self.robot.th) # go home

        #debugging
        print(f"Pose: x={x:.2f}, y={y:.2f}, th={th:.2f}")
        print(f"Control Inputs: v={inputs['linear_velocity']:.2f}, w={inputs['angular_velocity']:.2f}")
        print(f"Duty Cycles: left={duty_cycle_l:.2f}, right={duty_cycle_r:.2f}")
        
        
        # Log data
        # Here, I record and log everything so we can graph it.
        poses.append([x,y,th])
        duty_cycle_commands.append([duty_cycle_l,duty_cycle_r])
        velocities.append([robot.wl,robot.wr])
        desired_velocities.append([wl_desired, wr_desired])
        count += 1
        time.sleep(robot.dt)
        print(count)
        
        break

except KeyboardInterrupt:
    print("Stopping robot")
    robot.set_motor_speed(0, 0)

    duration = time.time() - start_time

    # Plot robot data
#     display.clear_output(wait=True)
    fig = plt.figure(figsize=(15,9))
    
    plt.subplot(2, 2, 1)
    plt.plot(np.array(poses)[:,0],np.array(poses)[:,1], 'b', label='Robot Path')
    plt.plot(np.array(goal_positions)[:,0],np.array(goal_positions)[:,1],'ro', label='Goal Position')
    plt.plot(x,y,'k',marker='+')
    plt.quiver(x,y,0.1*np.cos(th),0.1*np.sin(th))
    plt.xlabel('x-position (m)')
    plt.xticks(np.arange(0, 5, 0.5))
    plt.yticks(np.arange(0, 5, 0.5))
    plt.ylabel('y-position (m)')
    plt.title(f"Ran for {duration:.2f} secs")
#     plt.axis('equal')
    plt.grid()

    plt.subplot(2, 2, 2)
    duty_cycle_commands = np.array(duty_cycle_commands)
    plt.plot(duty_cycle_commands[:, 0], label='Left Wheel')
    plt.plot(duty_cycle_commands[:, 1], label='Right Wheel')
    plt.xlabel('Time Step')
    
    plt.ylabel('Duty Cycle')
    plt.title('Duty Cycle Commands Over Time')
    plt.legend() 
    plt.grid()
    
    plt.subplot(2, 2, 3)
    velocities = np.array(velocities)
    desired_velocities = np.array(desired_velocities)
    plt.plot(velocities[:, 0], label='Left Wheel')
    plt.plot(velocities[:, 1], label='Right Wheel')
    plt.plot(desired_velocities[:, 0], label='Desired Left Wheel')
    plt.plot(desired_velocities[:, 1], label='Desired Right Wheel')
    plt.xlabel('Time Step')
    plt.ylabel('Wheel Velocity (rad/s)')
    plt.title('Wheel Velocity vs. Time')
    plt.legend() 
    plt.grid()
   
        
    fig.tight_layout()

    display.display(plt.gcf())

#     GPIO.cleanup()

    """
    Hopefully the graphs are self explanatory. If not, please ask me. I'm happy to explain anything.
    """
